In [1]:
# DFS running combat

In [2]:
import multiprocessing

In [3]:
from DFS_World_States import world, world_grid_states, Weapon, Potion, Shield, Wand
from DFS_Universal_Rules import entity, theoretical_turn_length, subaction_dict, target_distance_scores, fire_bolt, ray_of_frost, shield
from DFS_Action_Series import RuleBasedSequenceDFS, action_rules 
from DFS_Location_Series import RuleBasedLocationSequenceDFS2, location_rules
from DFS_Functions import precalc_reward_series, analyze_reward_distribution, post_loc_series_reward_calc, post_obj_reward_series_calc, post_obj_reward_series_calc2
from DFS_Object_Series import RuleBasedObjectSequenceDFS1, object_rules

In [4]:
stage = world(11)

stage.generate_map()

actor = entity(stage)
actor.add_spell(fire_bolt)
actor.add_spell(ray_of_frost)
actor.add_spell(shield)

stage.add_enemy((1,1))
stage.add_enemy((-2,0))
stage.add_enemy((0,3))
#stage.add_enemy((2,2))
#stage.add_enemy((-3,-3))
#stage.add_enemy((1,-1))

stage.add_coin((3,0))
stage.add_coin((-1,-1))
stage.add_coin((1,1))

stage.add_item_to_inventory(actor, Weapon('dagger1', 'weapon', 1, 1, 0))
stage.add_item_to_inventory(actor, Weapon('dagger2', 'weapon', 1, 1, 0))
stage.add_item_to_inventory(actor, Weapon('greataxe', 'weapon', 1, 1, 0))
stage.add_item_to_inventory(actor, Shield('shield','shield', 1))
stage.add_item_to_inventory(actor, Wand('wand','wand', 1))


stage.add_weapon_dagger((-2,-1))
stage.add_weapon_dagger((-1,-2))
stage.add_weapon_greataxe((2,0))


In [5]:
#stage.grid2

In [6]:
action_sequence_generator = RuleBasedSequenceDFS(min_length=0,max_length=theoretical_turn_length, start=0, end=len(subaction_dict), rules=action_rules, acting_entity=actor)
action_series_full_list = action_sequence_generator.generate_sequences()
#action_series_full_list

#action_sequence_generator = ParallelizedRuleBasedSequenceDFS(min_length=0, max_length=theoretical_turn_length, start=0, end=len(subaction_dict), rules=action_rules, acting_entity=actor)
#action_series_full_list = action_sequence_generator.generate_sequences()
#action_series_full_list




In [7]:
reward_series_full_list = precalc_reward_series(action_series_full_list, actor)
#reward_series_full_list

In [8]:
analyze_reward_distribution(reward_series_full_list, action_series_full_list)

Total count: 1442210
High: 8
Low: -2

Reward Distribution:
% of -2s: 0.08%
% of -1s: 1.12%
% of 0s: 6.22%
% of 1s: 16.29%
% of 2s: 32.19%
% of 3s: 28.40%
% of 4s: 11.68%
% of 5s: 3.31%
% of 6s: 0.66%
% of 7s: 0.05%
% of 8s: 0.00%

Quality Threshold: 4.0
Total Qualifiers: 226424



In [9]:
location_series_generator = RuleBasedLocationSequenceDFS2(action_series_list=action_series_full_list,
                                                             reward_series_list=reward_series_full_list,
                                                             location_rules=location_rules,
                                                             acting_entity=actor,
                                                             target_distance_scores=target_distance_scores)
location_series_full_list = location_series_generator.generate_sequences()

In [10]:
post_location_reward_list = post_loc_series_reward_calc(action_series_full_list, location_series_full_list, reward_series_full_list, actor)

In [13]:
print(len(post_location_reward_list))
#post_location_reward_list

551029


In [12]:
# Object Series
object_series_generator = RuleBasedObjectSequenceDFS1(post_location_reward_list = post_location_reward_list, 
                                                     target_distance_scores = target_distance_scores,
                                                     object_rules = object_rules, 
                                                     acting_entity = actor)
object_series_full_list = object_series_generator.generate_object_sequences()

IndexError: list index out of range

In [ ]:


#print(len(object_series_full_list))

#object_series_full_list

#for i in object_series_full_list:
#    for j in i:
#      print(j)

#    print('\n')

In [ ]:
# I want the number of items in the object_series_full_list that are not empty
len([i for i in object_series_full_list if i != []])

In [ ]:
#post_location_reward_list

In [ ]:
#object_series_full_list

In [16]:
#act_loc_obj_rew = create_act_loc_obj_rew(post_location_reward_list, object_series_full_list, actor)
#act_loc_obj_rew

In [ ]:
post_object_reward_list = post_obj_reward_series_calc2(object_series_full_list, actor)
post_object_reward_list

In [18]:
#post_object_reward_list = post_obj_reward_series_calc(action_series_full_list = action_series_full_list, 
#                                                      location_series_full_list = location_series_full_list, 
#                                                      object_series_full_list = object_series_full_list, 
#                                                      reward_series_full_list = reward_series_full_list, 
#                                                      acting_entity = actor)

# this one will include how much damage is being dealt
# if an effective weapon is equipped
# how many coins are picked up

In [19]:
# entity series list???
# in order to specify which entity is being targeted

# shortcuts:
# - currently based on the location series, if there is only one entity in a space, the entity is automatically targeted
# - only if there are multiple entities in a space, the entity series is used to determine the target

In [20]:
# processing a round of combat that includes the queue of actions and possible reactions other entities take

In [21]:
# what needs to happen before I can be running combat?
# - the function to process a creature's turn needs to be done
# - the function to process everyone else's reaction needs to be operational
# - enemies need to be able to take their turn, currently they operate as a form of static entity



In [22]:
# what needs to happen before the next stage?
# - combat situations needs to be randomly generated
# - combat scenarios need to be ran millions of times